In [1]:
####Packages####
from Bio import SeqIO
from BCBio.GFF import GFFExaminer
from Bio.Seq import Seq
from Bio.SeqFeature import SeqFeature, FeatureLocation
from Bio.SeqRecord import SeqRecord
import pandas as pd
import numpy as np
import gff3_parser
import glob
import numpy as np
import os
from pathlib import Path
import pprint
import re
import shutil
import subprocess

In [2]:
#Pull in the Genome Annotation File for the correct species & Parse
os.chdir('/home/jake/Downloads/Genomes/gff_files/')
anotes=os.listdir('/home/jake/Downloads/Genomes/gff_files/')
for f in anotes:
    gff=os.path.basename(f)
    name=gff.split('.')[0] #Removes.gff
    gname=name
    _ = gff3_parser.parse_gff3(gff,verbose = False,  parse_attributes=True)
    df=_[['Seqid', 'Type', 'Start', 'End', 'Strand','gene', 'gene_biotype']]
    df.index.name='Index'
    df=df.loc[(df['Type']=='gene')& (df['gene_biotype']=='protein_coding')]
    df=df.astype({'Start':int,'End':int})
    #Create Filtered Gene Dataframes
    all_df=df.loc[(df['Type']=='gene') & (df['gene'].str.contains(r'hsp', re.I))] # ALL HSP genes
    neg_df=df.loc[(df['Strand']=='-')& (df['Type']=='gene') & (df['gene'].str.contains(r'hsp', re.I))] #Negative HSP
    pos_df=df.loc[(df['Strand']=='+')& (df['Type']=='gene') & (df['gene'].str.contains(r'hsp', re.I))] #Positive HSP
    alen=len(all_df)
    nlen=len(neg_df)
    plen=len(pos_df)
    print(f"{name} Genome imported")
    print(f"There are {alen} total hsp genes found in genome")
    print(f"{nlen} hsp genes are found on the negative strand")
    print(f"{plen} hsp genes are found on the positive strand")

Anguilla_anguilla Genome imported
There are 22 total hsp genes found in genome
8 hsp genes are found on the negative strand
14 hsp genes are found on the positive strand


In [3]:
#Annotated Gene Strands DF's
nstrand=df.loc[(df['Strand']=='-')].sort_values(by="Start", ascending=True)#Negative Strand Genes
#master_nstrand=df.loc[(df['Strand']=='-')].sort_values(by="Start", ascending=True)
pstrand=df.loc[(df['Strand']=='+')].sort_values(by="Start", ascending=True)#Negative Strand Genes
#List of Columns to manipulate
clist=['shift_1', 'shift_2', 'shift_3', 'shift_4', 'shift_5', 'shift_6', 'shift_7', 'shift_8', 'shift_9', 'shift_10']
print(f"Negative Strands Created Sucessfully")
print(f"Positive Strands Created Sucessfully")

Negative Strands Created Sucessfully
Positive Strands Created Sucessfully


In [4]:
#Function  manipulate the dataframes
def seq_manipulate(df):
    df['shift_1']=df['Start']- df['End'].shift(+1)
    df['shift_2']=df['Start']- df['End'].shift(+2)
    df['shift_3']=df['Start']- df['End'].shift(+3)
    df['shift_4']=df['Start']- df['End'].shift(+4)
    df['shift_5']=df['Start']- df['End'].shift(+5)
    df['shift_6']=df['Start']- df['End'].shift(+6)
    df['shift_7']=df['Start']- df['End'].shift(+7)
    df['shift_8']=df['Start']- df['End'].shift(+8)
    df['shift_9']=df['Start']- df['End'].shift(+9)
    df['shift_10']=df['Start']- df['End'].shift(+10)
    df[clist]= df[clist].clip(lower=0)#Remove all of the negative values and convert them to 0
    df[clist]= df[clist].fillna(0) #Convert NA values to 0
    df[clist]= df[clist].replace(0, np.nan) #Convert all the 0 values to NA
    df['inter_genic']=df[clist].min(1)
    df['Stop']= df['Start'] + df['inter_genic']
    df['combined']=df.apply(lambda x:'%s:%s'%(x['Start'],x['End']),axis=1)
    #df.update('"' + df['combined'].astype(str) + '"')
#Transfrom both strands of DFs

seq_manipulate(nstrand)
print(f"Negative Strand Manipulated Sucessfully")
seq_manipulate(pstrand)
print(f"Positive Strand Manipulated Sucessfully")
#Create Two Filtered Data Sets
n_=nstrand[['Seqid', 'Type', 'Strand','gene', 'gene_biotype', 'Start', 'Stop', 'combined']]
p_=pstrand[['Seqid', 'Type', 'Strand','gene', 'gene_biotype', 'Start', 'Stop', 'combined']]
n_hsp=n_.loc[(n_['gene'].str.contains(r'hsp', re.I))] # ALL HSP genes
p_hsp=p_.loc[(p_['gene'].str.contains(r'hsp', re.I))] # ALL HSP genes
hsps= pd.concat([n_hsp,p_hsp])
hsps=hsps.sort_values(by="Start", ascending=True)
hsps['Chromosome']= hsps['Seqid'].apply(lambda x: x[-4:-2])
export=hsps[['gene', 'combined', 'Chromosome', 'Start', 'Stop']]
x=export.reset_index()
x=x[['gene', 'combined', 'Chromosome', 'Start', 'Stop']]
x.to_csv(f"/home/jake/Downloads/{name}.csv")
print(f"Chromosome Column Created")

Negative Strand Manipulated Sucessfully
Positive Strand Manipulated Sucessfully
Chromosome Column Created


In [5]:
#Need to copy the genome directory and remove the line wraps(/n)
source_folder=(f"/home/jake/Downloads/Genomes/{name}/")
destination_folder= (f"/home/jake/Downloads/Genomes/{name}_noninterleaved/")
path = (f"/home/jake/Downloads/Genomes/{name}_noninterleaved/")
# Check whether the specified path exists or not
isExist = os.path.exists(path)
if not isExist:
  # Create a new directory because it does not exist 
  os.makedirs(path)
  print("The new directory is created!")
for file_name in os.listdir(source_folder):
    # construct full file path
    source = source_folder + file_name
    destination = destination_folder + file_name
    # copy only files
    if os.path.isfile(source):
        shutil.copy(source, destination)
        print('copied', file_name)

print(f"Now open terminal, cd into downloads and run ./for_loop_updated.sh for {name}")

copied chr10.fna
copied chr11.fna
copied chr12.fna
copied chr13.fna
copied chr14.fna
copied chr15.fna
copied chr16.fna
copied chr17.fna
copied chr18.fna
copied chr19.fna
copied chr01.fna
copied chr02.fna
copied chr03.fna
copied chr04.fna
copied chr05.fna
copied chr06.fna
copied chr07.fna
copied chr08.fna
copied chr09.fna
Now open terminal, cd into downloads and run ./for_loop_updated.sh for Anguilla_anguilla


In [6]:
#Going to attempt to read in each sequence, parse it into a filename and seq:
folder_path = path
fasta_paths = glob.glob(os.path.join(folder_path, '*.fna'))
seq=[]
names=[]
for fasta_path in fasta_paths:
    names.append(os.path.basename(fasta_path))
    for seq_record in SeqIO.parse(fasta_path, "fasta"):
        seq.append(seq_record.seq)
fasta_dict={names[i]:seq[i] for i in range(len(names))}
        

In [7]:
#Itterate through the df
slice= []
for f in x['Chromosome']:
    for name in fasta_dict:
        if f in name:
            value_pair=(fasta_dict.get(name))
            slice.append(value_pair)
            #print(value_pair[0:10])
            x['Chr']=value_pair
x['Stop'] = x['Stop'].astype(int)
x['new_col'] = x.apply(lambda x: x['Chr'][x['Start']-1:x['Stop']], 1)
#x['new_col']=str(seq(x['new_col']))
x['len']= x['new_col'].str.len()
x['header']=x['gene']
we_did_it=dict(zip(x["header"], x["new_col"]))
x['new_col']=x['new_col'].astype(str)
#display(x['new_col'])
formatted=[]
for i in we_did_it.values():
    v = str(i)
    formatted.append(v)
x['format']=formatted
fml=dict(zip(x["header"], x["format"]))

        

In [8]:
destination_folder= (f"/home/jake/Downloads/Genomes/intergenic_exports/{gname}")
path = (f"/home/jake/Downloads/Genomes/intergenic_exports/{gname}")
# Check whether the specified path exists or not
isExist = os.path.exists(path)
if not isExist:
  # Create a new directory because it does not exist 
  os.makedirs(path)
  print("The new directory is created!")
os.chdir(path)
with open(f'{gname}_clean.fasta', 'w') as output_file:
    for name, seq in fml.items():
        output_file.write(f">{gname}_{name}\n{seq}\n")

print(f"Exported fasta sequence file is in {destination_folder}")

Exported fasta sequence file is in /home/jake/Downloads/Genomes/intergenic_exports/Anguilla_anguilla


In [28]:
print(os.getcwd())

/home/jake/Downloads/Genomes/intergenic_exports/Anguilla_anguilla


In [49]:
#Open Command line and input this command
##
##cat ______clean.fasta | awk '{
        if (substr($0, 1, 1)==">") {filename=(substr($0,2) ".fasta")}
        print $0 >> filename
        close(filename)
}'

In [63]:
##Change directory into repeat masker and run the script
lib_path='/home/jake/Downloads/Genomes/FishTEDB/'
lib=(f"'/home/jake/Downloads/Genomes/FishTEDB/'{gname}_lib.fa")
#See GFF_Annotater for the genome and gene variables
destz=(f'/home/jake/Downloads/Genomes/intergenic_exports/{gname}/')
os.chdir('/home/jake/Downloads/Bio_tools/RepeatMasker-4.1.2-p1/RepeatMasker/')
anotez=os.listdir(f'/home/jake/Downloads/Genomes/intergenic_exports/{gname}/')
for w in anotez:
    proximal_promoter=destz+w
    os.system(f"./RepeatMasker -lib {lib} {proximal_promoter}")

RepeatMasker version 4.1.2-p1
Search Engine: NCBI/RMBLAST [ 2.11.0+ ]
Using Custom Repeat Library: /home/jake/Downloads/Genomes/FishTEDB/Anguilla_anguilla_lib.fa


analyzing file /home/jake/Downloads/Genomes/intergenic_exports/Anguilla_anguilla/Anguilla_anguilla_clean.fasta

Checking for E. coli insertion elements
identifying Simple Repeats in batch 1 of 2
identifying matches to Anguilla_anguilla_lib.fa sequences in batch 1 of 2
identifying Simple Repeats in batch 1 of 2

Checking for E. coli insertion elements
identifying Simple Repeats in batch 2 of 2
identifying matches to Anguilla_anguilla_lib.fa sequences in batch 2 of 2
identifying Simple Repeats in batch 2 of 2
processing output: 
cycle 1 
cycle 2 
cycle 3 
cycle 4 
cycle 5 
cycle 6 
cycle 7 
cycle 8 
cycle 9 
cycle 10 
Generating output... 
masking
done
RepeatMasker version 4.1.2-p1
Search Engine: NCBI/RMBLAST [ 2.11.0+ ]
Using Custom Repeat Library: /home/jake/Downloads/Genomes/FishTEDB/Anguilla_anguilla_lib.fa


analyzing fil

In [69]:
source_folderz = (f'/home/jake/Downloads/Genomes/intergenic_exports/{gname}/')
destination_folderz = (f'/home/jake/Downloads/repeat_outputs/{gname}/')
isExist = os.path.exists(destination_folderz)
if not isExist:
  # Create a new directory because it does not exist 
  os.makedirs(destination_folderz)
  print("The new directory is created!")
outputz=os.listdir(source_folderz)
# iterate files
for f in outputz:
    if f.endswith(('cat','tbl','all','masked','out')):
        src=source_folderz+f
        dest=destination_folderz+f
        shutil.move(src, dest)
        print("Moved: ", f)
    else:
        pass

Moved:  Anguilla_anguilla_carhsp1.fasta.cat
Moved:  Anguilla_anguilla_carhsp1.fasta.masked
Moved:  Anguilla_anguilla_carhsp1.fasta.out
Moved:  Anguilla_anguilla_carhsp1.fasta.tbl
Moved:  Anguilla_anguilla_clean.fasta.cat
Moved:  Anguilla_anguilla_clean.fasta.masked
Moved:  Anguilla_anguilla_clean.fasta.out
Moved:  Anguilla_anguilla_clean.fasta.tbl
Moved:  Anguilla_anguilla_hsp90ab1.fasta.cat
Moved:  Anguilla_anguilla_hsp90ab1.fasta.masked
Moved:  Anguilla_anguilla_hsp90ab1.fasta.out
Moved:  Anguilla_anguilla_hsp90ab1.fasta.tbl
Moved:  Anguilla_anguilla_hsp90b1.fasta.cat
Moved:  Anguilla_anguilla_hsp90b1.fasta.masked
Moved:  Anguilla_anguilla_hsp90b1.fasta.out
Moved:  Anguilla_anguilla_hsp90b1.fasta.tbl
Moved:  Anguilla_anguilla_hspa1b.fasta.cat
Moved:  Anguilla_anguilla_hspa1b.fasta.masked
Moved:  Anguilla_anguilla_hspa1b.fasta.out
Moved:  Anguilla_anguilla_hspa1b.fasta.tbl
Moved:  Anguilla_anguilla_hspa4a.fasta.cat
Moved:  Anguilla_anguilla_hspa4a.fasta.masked
Moved:  Anguilla_anguill